In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.0 MB/s eta 0:00:00


## Download en & ja dataset

In [ ]:
# target text
text_en = """This paper describes SentencePiece, a
language-independent subword tokenizer
and detokenizer designed for Neural-based
text processing, including Neural Machine
Translation.""".replace("\n", " ")

text_ja = "この文書では、ニューラル機械翻訳を含むニューラルベースのテキスト処理用に設計された、言語に依存しないサブワードトークナイザーおよびデトークナイザーである SentencePiece について説明します。"
text_ko = "이 문서에서는 신경망 기계 번역을 포함하여 신경망 기반 텍스트 처리용으로 설계된 언어 독립적 하위 단어 토크나이저 및 디토큰나이저인 SentencePiece에 대해 설명합니다."

In [ ]:
from tqdm import tqdm
from datasets import load_dataset

dataset = load_dataset("opus100", "en-ja")

with open("opus100-en.txt", "w") as f_en, open("opus100-ja.txt", "w") as f_ja:
    for it in dataset["train"]:
        f_en.write(it["translation"]["en"] + "\n")
        f_ja.write(it["translation"]["ja"] + "\n")


  0%|          | 0/3 [00:00<?, ?it/s]

# Usage in transformers

## WordPiece on English

In [ ]:
# https://huggingface.co/learn/nlp-course/chapter6/8?fw=pt#building-a-wordpiece-tokenizer-from-scratch
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
    Regex,
)
import time
import os
from copy import deepcopy

In [ ]:
_text = deepcopy(text_en)
print(_text)

#normalizers.Sequence([normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()])
normalizer = normalizers.BertNormalizer()

_text = normalizer.normalize_str(_text)
print(_text)


pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(),
                                         pre_tokenizers.Digits(individual_digits=True)])
_text = pre_tokenizer.pre_tokenize_str(_text)
print(_text)


post_processor = processors.TemplateProcessing(
    single="[CLS] $A [EOS]",
    special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
)

def build_wordpiece_tokenizer():
    tokenizer = Tokenizer(model = models.WordPiece(unk_token="[UNK]"))
    tokenizer.normalizer = normalizers.Sequence([normalizers.NFD(),
                                                 normalizers.Lowercase(),
                                                 normalizers.StripAccents()])
    tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Whitespace(),
                                                       pre_tokenizers.Digits(individual_digits=True)])
    tokenizer.post_processor = processors.TemplateProcessing(
        single="[CLS] $A [EOS]",
        special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
    )
    return tokenizer
tokenizer = build_wordpiece_tokenizer()

This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.
this paper describes sentencepiece, a language-independent subword tokenizer and detokenizer designed for neural-based text processing, including neural machine translation.
[('this', (0, 4)), ('paper', (5, 10)), ('describes', (11, 20)), ('sentencepiece', (21, 34)), (',', (34, 35)), ('a', (36, 37)), ('language', (38, 46)), ('-', (46, 47)), ('independent', (47, 58)), ('subword', (59, 66)), ('tokenizer', (67, 76)), ('and', (77, 80)), ('detokenizer', (81, 92)), ('designed', (93, 101)), ('for', (102, 105)), ('neural', (106, 112)), ('-', (112, 113)), ('based', (113, 118)), ('text', (119, 123)), ('processing', (124, 134)), (',', (134, 135)), ('including', (136, 145)), ('neural', (146, 152)), ('machine', (153, 160)), ('translation', (161, 172)), ('.', (172, 173))]


In [ ]:
tokenizer.get_vocab()

{}

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

tokenizer.train(files=["opus100-en.txt"], trainer=trainer)

os.makedirs("./tokenizers", exist_ok=True)
tokenizer.save("./tokenizers/en-wordpiece.json")

KeyboardInterrupt: ignored

In [ ]:
for i, (key, value) in enumerate(tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)


steven 6486
##eds 2073
fired 4042
stubborn 8083
creator 8699
organized 9130
americ 1741
comic 9654
##ual 1399
vis 1411
gro 4999
tox 8669
##ning 802
cd 4874
th 382
� 226
preparing 8356
merle 9926
become 1797
ps 8785


In [ ]:
encoded = tokenizer.encode(text_en)
print("len:", len(encoded.tokens), encoded.tokens)

print(tokenizer.decode(encoded.ids))
print("------------------")

print(tokenizer.normalizer.normalize_str(text_en))
print(tokenizer.decode(encoded.ids).replace(" ##", ""))

len: 40 ['[CLS]', 'this', 'paper', 'describe', '##s', 'sentence', '##pi', '##ec', '##e', ',', 'a', 'language', '-', 'independent', 'sub', '##word', 'to', '##ken', '##izer', 'and', 'det', '##oken', '##izer', 'designed', 'for', 'neur', '##al', '-', 'based', 'text', 'process', '##ing', ',', 'including', 'neur', '##al', 'machine', 'translation', '.', '[EOS]']
this paper describe ##s sentence ##pi ##ec ##e , a language - independent sub ##word to ##ken ##izer and det ##oken ##izer designed for neur ##al - based text process ##ing , including neur ##al machine translation .
------------------
this paper describes sentencepiece, a language-independent subword tokenizer and detokenizer designed for neural-based text processing, including neural machine translation.
this paper describes sentencepiece , a language - independent subword tokenizer and detokenizer designed for neural - based text processing , including neural machine translation .


In [ ]:
encoded = tokenizer.encode(text_ko)
print("len:", len(encoded.tokens), encoded.tokens)

encoded = tokenizer.encode(text_ja)
print("len:", len(encoded.tokens), encoded.tokens)

len: 24 ['[CLS]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '.', '[EOS]']
len: 13 ['[CLS]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', 'sentence', '##pi', '##ec', '##e', '[UNK]', '[UNK]', '[EOS]']


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from transformers import BertTokenizerFast
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

text_en = "hello-world!"
encoded = bert_tokenizer.encode(text_en)
print("len:", len(encoded))

print(text_en)
print(bert_tokenizer.decode(encoded))
# note again `language-independent` -> `language - independent`

len: 6
hello-world!
[CLS] hello - world! [SEP]


## Sentencepiece on English

In [ ]:
_text = deepcopy(text_en)
print(_text)

normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "), # replace 2+ spaces with one.
    ]
)


_text = normalizer.normalize_str(_text)
print(_text)


pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Metaspace(),
                                         pre_tokenizers.Digits(individual_digits=True)])
_text = pre_tokenizer.pre_tokenize_str(_text)
print(_text) # unlike `Whitespace`, `Metaspace` preserves space with '_'.


post_processor = processors.TemplateProcessing(
    single="[CLS] $A [EOS]",
    special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
)

def build_sentencepiece_tokenizer():
    tokenizer = Tokenizer(model = models.WordPiece(unk_token="[UNK]"))
    tokenizer.normalizer = normalizers.Sequence(
        [
            normalizers.Replace("``", '"'),
            normalizers.Replace("''", '"'),
            normalizers.NFKD(),
            normalizers.StripAccents(),
            normalizers.Replace(Regex(" {2,}"), " "), # replace 2+ spaces with one.
        ]
    )
    tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Metaspace(),
                                                       pre_tokenizers.Digits(individual_digits=True)])
    tokenizer.post_processor = processors.TemplateProcessing(
        single="[CLS] $A [EOS]",
        special_tokens=[("[CLS]", 1), ("[EOS]", 2)],
    )
    return tokenizer
tokenizer = build_sentencepiece_tokenizer()

This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.
This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.
[('▁This', (0, 4)), ('▁paper', (4, 10)), ('▁describes', (10, 20)), ('▁SentencePiece,', (20, 35)), ('▁a', (35, 37)), ('▁language-independent', (37, 58)), ('▁subword', (58, 66)), ('▁tokenizer', (66, 76)), ('▁and', (76, 80)), ('▁detokenizer', (80, 92)), ('▁designed', (92, 101)), ('▁for', (101, 105)), ('▁Neural-based', (105, 118)), ('▁text', (118, 123)), ('▁processing,', (123, 135)), ('▁including', (135, 145)), ('▁Neural', (145, 152)), ('▁Machine', (152, 160)), ('▁Translation.', (160, 173))]


In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

tokenizer.train(files=["opus100-en.txt"], trainer=trainer)

os.makedirs("./tokenizers", exist_ok=True)
tokenizer.save("./tokenizers/en-sentpiece.json")

In [ ]:
for i, (key, value) in enumerate(tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)


▁dad.
 5373
▁loved 3208
▁serious?
 8232
▁word, 8574
▁Hanna 7253
▁chast 5674
##ighty, 9846
▁guy.
 2827
▁Wall 6379
▁destroy 2328
##pt 804
##year-old 5962
▁jealous 8613
‭ 180
▁liar 6457
▁a...
 6358
▁Home 5006
▁fancy 9669
##unt 2516
▁island 3628


In [ ]:
encoded = tokenizer.encode(text_en)
print("len:", len(encoded.tokens), encoded.tokens)

print(tokenizer.decode(encoded.ids))
print("------------------")

print(tokenizer.normalizer.normalize_str(text_en))
print(tokenizer.decode(encoded.ids).replace("##", "").replace(" ", "")[1:].replace('▁', ' '))

len: 49 ['[CLS]', '▁This', '▁paper', '▁describe', '##s', '▁Sen', '##ten', '##ce', '##P', '##iec', '##e,', '▁a', '▁language', '##-in', '##de', '##pen', '##den', '##t', '▁sub', '##word', '▁to', '##ken', '##ize', '##r', '▁and', '▁det', '##oke', '##n', '##ize', '##r', '▁designed', '▁for', '▁Ne', '##ural', '##-b', '##ased', '▁text', '▁process', '##ing,', '▁including', '▁Ne', '##ural', '▁Mac', '##h', '##ine', '▁Trans', '##l', '##ation.', '[EOS]']
▁This ▁paper ▁describe ##s ▁Sen ##ten ##ce ##P ##iec ##e, ▁a ▁language ##-in ##de ##pen ##den ##t ▁sub ##word ▁to ##ken ##ize ##r ▁and ▁det ##oke ##n ##ize ##r ▁designed ▁for ▁Ne ##ural ##-b ##ased ▁text ▁process ##ing, ▁including ▁Ne ##ural ▁Mac ##h ##ine ▁Trans ##l ##ation.
------------------
This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.
This paper describes SentencePiece, a language-independent subword tokenizer and deto

In [ ]:
from tokenizers import Tokenizer
from transformers import XLNetTokenizer
xlnet_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

encoded = xlnet_tokenizer.encode(text_en)
print("len:", len(encoded))

print(text_en)
print(xlnet_tokenizer.decode(encoded))


len: 38
This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.
This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.<sep><cls>


In [ ]:
import sentencepiece as spm
t1 = time.time()
_ = spm.SentencePieceTrainer.train(
    input='opus100-en.txt',
    model_prefix='en-sp',
    model_type="bpe",
    vocab_size=10000,
    )
print("en-sentpiece google time:", time.time() - t1)


en-sentpiece google time: 36.54629874229431


In [ ]:
sp = spm.SentencePieceProcessor(model_file='./en-sp.model')

encoded = sp.encode(text_en)
print("len:", len(encoded), encoded)
print("------------------")
print(text_en)
print(sp.decode(encoded))

len: 45 [285, 2419, 3167, 7344, 9532, 575, 9964, 6664, 9940, 6, 3684, 9948, 199, 6669, 110, 1269, 3506, 26, 1678, 8024, 67, 1304, 9922, 1678, 8024, 4885, 89, 1566, 6142, 9948, 9941, 2950, 2373, 2787, 25, 9940, 5054, 1566, 6142, 8391, 293, 4956, 9929, 233, 9932]
------------------
This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.
This paper describes SentencePiece, a language-independent subword tokenizer and detokenizer designed for Neural-based text processing, including Neural Machine Translation.


In [ ]:
wp_tokenizer = build_wordpiece_tokenizer()
sp_tokenizer = build_sentencepiece_tokenizer()

special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

t1 = time.time()
wp_tokenizer.train(files=["opus100-en.txt"], trainer=trainer)
print("en-wordpiece time:", time.time() - t1)

t1 = time.time()
sp_tokenizer.train(files=["opus100-en.txt"], trainer=trainer)
print("en-sentpiece time:", time.time() - t1)

os.makedirs("./tokenizers", exist_ok=True)
wp_tokenizer.save("./tokenizers/en-wordpiece.json")
sp_tokenizer.save("./tokenizers/en-sentpiece.json")

en-wordpiece time: 22.483790159225464
en-sentpiece time: 50.391443967819214


## WordPiece & sentencepiece on Japanese

In [ ]:
wp_tokenizer = build_wordpiece_tokenizer()
sp_tokenizer = build_sentencepiece_tokenizer()

special_tokens = ["[UNK]", "[PAD]", "[CLS]"]
trainer = trainers.WordPieceTrainer(vocab_size=10000, special_tokens=special_tokens, )

import time
t1 = time.time()
wp_tokenizer.train(files=["opus100-ja.txt"], trainer=trainer)
print("ja-wordpiece time:", time.time() - t1)

t1 = time.time()
sp_tokenizer.train(files=["opus100-ja.txt"], trainer=trainer)
print("ja-sentpiece time:", time.time() - t1)

os.makedirs("./tokenizers", exist_ok=True)
wp_tokenizer.save("./tokenizers/ja-wordpiece.json")
sp_tokenizer.save("./tokenizers/ja-sentpiece.json")

ja-wordpiece time: 102.06447887420654
ja-sentpiece time: 202.78346800804138


In [ ]:
encoded = wp_tokenizer.encode(text_ja)
print("len:", len(encoded.tokens), encoded.tokens)

print(wp_tokenizer.decode(encoded.ids))
print("------------------")

encoded = sp_tokenizer.encode(text_ja)
print("len:", len(encoded.tokens), encoded.tokens)
print(sp_tokenizer.decode(encoded.ids))

len: 73 ['[CLS]', 'この', '##文', '##書', '##ては', '、', 'ニュー', '##ラ', '##ル', '##機', '##械', '##翻', '##訳', '##を', '##含', '##む', '##ニ', '##ュー', '##ラ', '##ル', '##ヘース', '##のテ', '##キ', '##スト', '##処', '##理', '##用', '##に', '##設', '##計', '##された', '、', '言', '##語', '##に', '##依', '##存', '##しない', '##サ', '##フ', '##ワー', '##ト', '##ト', '##ーク', '##ナ', '##イ', '##サー', '##お', '##よ', '##ひ', '##テ', '##ト', '##ーク', '##ナ', '##イ', '##サー', '##てある', 'se', '##nt', '##en', '##c', '##e', '##p', '##i', '##e', '##c', '##e', 'に', '##ついて', '##説明', '##します', '。', '[EOS]']
この ##文 ##書 ##ては 、 ニュー ##ラ ##ル ##機 ##械 ##翻 ##訳 ##を ##含 ##む ##ニ ##ュー ##ラ ##ル ##ヘース ##のテ ##キ ##スト ##処 ##理 ##用 ##に ##設 ##計 ##された 、 言 ##語 ##に ##依 ##存 ##しない ##サ ##フ ##ワー ##ト ##ト ##ーク ##ナ ##イ ##サー ##お ##よ ##ひ ##テ ##ト ##ーク ##ナ ##イ ##サー ##てある se ##nt ##en ##c ##e ##p ##i ##e ##c ##e に ##ついて ##説明 ##します 。
------------------
len: 76 ['[CLS]', '▁この', '##文', '##書', '##ては', '##、', '##ニ', '##ュー', '##ラ', '##ル', '##機', '##械', '##翻', '##訳', '##を', '##含', '##む', '##ニ', '##ュー', '#

In [ ]:
print("- Wordpiece --------")
for i, (key, value) in enumerate(wp_tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)

print("- Sentencepiece --------")
for i, (key, value) in enumerate(sp_tokenizer.get_vocab().items()):
    if i == 20:
        break
    print(key, value)


- Wordpiece --------
##鏡 5181
寛 1099
悛 1347
时 1646
蝶 2950
##しゃない 7779
##ことか 7912
##or 8007
日 1639
県 2341
△ 166
 3773
楓 1823
##仏 6464
##もう 8181
湯 2032
##護 4747
##/ 5875
鉾 3395
##耕 6747
- Sentencepiece --------
##しています
 9933
鞄 3554
纒 2648
##工 4846
##伍 6514
##F 4773
##騰 5556
稟 2447
##かし 8433
##のお 8890
▁第 9548
##蕁 6438
##业 7572
##神 4091
▁まあ 8735
カ 286
懃 1412
淘 2002
##からの 8696
##重要 8910


## NFKC unicode normalization for Korean.

In [ ]:
build_sentencepiece_tokenizer

<function __main__.build_sentencepiece_tokenizer()>